In [ ]:
pip install -qr requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from copy import deepcopy as copy

In [ ]:
df = pd.read_csv('data/expert_opinions.csv')

experts_count = len(df)
experts_answers = {}
types = [int(i) for i in df.keys()]
options = set()
for i in range(experts_count):
    experts_answers.update({i: []})
    for k in df:
        answer = df[k][i]
        experts_answers[i].append(answer)

        options.add(answer)

print(f'Множество значений: {types}')
print(f'Термы: {options}')

In [ ]:
for i in types:
    print(f'{i:<3} ', end='')
print()
print('----' * len(types))
for k in experts_answers:
    for i in options:
        for j in experts_answers[k]:
            if i == j:
                print('+   ', end='')
            else:
                print('    ', end='')
        print(i)
    print('----' * len(types))

In [ ]:
membership_table = {j: [0 for i in range(len(types))] for j in options}
for k in experts_answers:
    for i in range(len(experts_answers[k])):
        answer = experts_answers[k][i]
        membership_table[answer][i] += 1

for i in types:
    print(f'{i:<5} ', end='')
print()
print('------' * len(types))
for k in membership_table:
    for i in membership_table[k]:
        print(f'{i:<5} ', end='')
    print()
    for i in membership_table[k]:
        print(f'{i / experts_count:<5} ', end='')
    print(k)
    print('------' * len(types))

for k in membership_table:
    plt.plot(
        types,
        [i / experts_count for i in membership_table[k]],
        label=k
    )
plt.legend()
plt.show()


In [ ]:
category_tree = json.load(open('data/category_tree.json'))

def print_category_tree(Node=category_tree, prefix=''):
    level_size = len(Node)
    for category in Node:
        level_size -= 1

        if level_size == 0:
            angle = '└──'
            new_prefix = f'{prefix}   '
        else:
            angle = '├──'
            new_prefix = f'{prefix}│  '
        
        print(f'{prefix}{angle}{category}')
        print_category_tree(Node[category], new_prefix)


print_category_tree()

In [ ]:
def find_categories_proximity(lhs, rhs):
    def find_category(category, Node=category_tree):
        assert Node is not None

        for i in Node:
            if category == i:
                return [category]
            else:
                in_sub_tree = find_category(category, Node[i])
                if in_sub_tree is not None:
                    return [i] + in_sub_tree
            
        return None
    
    lhs = find_category(lhs)
    rhs = find_category(rhs)

    assert lhs is not None and rhs is not None
    for i in range(len(min(lhs, rhs))):
        if lhs[i] != rhs[i]:
            return 1 / (len(lhs) - i + len(rhs) - i + 1)
        
    return 1

def find_cals_proximity(lhs, rhs):
    return min(lhs, rhs) / max(lhs, rhs)

def find_fat_proximity(lhs, rhs):
    return min(lhs, rhs) / max(lhs, rhs)

def find_is_vegan_proximity(lhs, rhs):
    return int(lhs == rhs)

def find_raw_proximity(lhs, rhs):
    method = 'Без обработки'
    return int((rhs == method) == (lhs == method))

def find_taste_proximity(lhs, rhs):
    lhs = lhs.split(' ')
    rhs = rhs.split(' ')

    common = max(list(set(lhs).intersection(rhs)), [])

    return len(common) / (len(lhs) + len(rhs) - len(common))

def clac_proximity(df, func):
    matrix = []
    for i in df:
        matrix.append([])
        for j in df:
            matrix[-1].append(func(i, j))

    return matrix

def merge_matrix(lhs, rhs, *other):
    for i in range(len(lhs)):
        for j in range(len(lhs[i])):
            lhs[i][j] += rhs[i][j]
    
    if len(other) != 0:
        merge_matrix(lhs, *other)

    return lhs

def down_matrix(matrix, coef=0.5):
    matrix = copy(matrix)

    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            matrix[i][j] = matrix[i][j] * coef

    return matrix

In [ ]:
df = pd.read_csv('data/dishes.csv')
taste = [i for i in df['Вкус']]
for i in range(len(taste)):
    taste[i] = taste[i].replace(';', ' ')
df['Вкус'] = taste
df

In [ ]:
matrix = clac_proximity(df['Вкус'], find_taste_proximity)

plt.matshow(matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
matrix_for_recomendation = merge_matrix(
    down_matrix(clac_proximity(df['Приготовления'], find_raw_proximity), coef=0.1),
    down_matrix(clac_proximity(df['Вегетарианское'], find_is_vegan_proximity), coef=0.1),
    down_matrix(clac_proximity(df['Категория'], find_categories_proximity), coef=0.3),
    down_matrix(clac_proximity(df['Жирность'], find_fat_proximity), coef=0.2),
    down_matrix(clac_proximity(df['Калорийность'], find_cals_proximity), coef=0.2),
    down_matrix(clac_proximity(df['Вкус'], find_taste_proximity), coef=0.1)
)

plt.matshow(matrix_for_recomendation, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
matrix = clac_proximity(df['Категория'], find_categories_proximity)

plt.matshow(matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
def get_recomendation(likes, dislikes):
    def find(name):
        return df.loc[df['Название'] == name]

    recomendation = [0.5 for i in range(len(df))]
    for i in likes:
        dish_i = find(i).index[0]
        recomendation[dish_i] = None
        for j in range(len(matrix_for_recomendation[dish_i])):
            if recomendation[j] is None:
                continue

            recomendation[j] += matrix_for_recomendation[dish_i][j] / len(likes) * 0.5

    for i in dislikes:
        dish_i = find(i).index[0]
        recomendation[dish_i] = None
        for j in range(len(matrix_for_recomendation[dish_i])):
            if recomendation[j] is None:
                continue

            recomendation[j] -= matrix_for_recomendation[dish_i][j] / len(dislikes) * 0.5

    result = df
    result['Рекомендация'] = recomendation
    result = result.sort_values(by=['Рекомендация'], ascending=False)
    
    return result

In [ ]:
def do_fliter(recomendation, category=None, cals=None, fat=None, taste=None, is_vegan=None, method=None):
    # from expert_options.csv
    term_to_fat = {
        'Не жирное': lambda x: int(x) <= 5,
        'Слегка жирное': lambda x: 5 < int(x) <= 10,
        'Жирное': lambda x: 10 < int(x) <= 18,
        'Очень жирное': lambda x: int(x) > 18,
    }

    if category is not None:
        recomendation = recomendation[recomendation['Категория'].apply(lambda x: x == category)]
    
    if cals is not None:
        if type(cals) is not tuple:
            is_max = True
        else:
            is_max = cals[1] == 'max'
            cals = cals[0]
        
        if is_max:
            recomendation = recomendation[recomendation['Калорийность'].apply(lambda x: x <= cals)]
        else:
            recomendation = recomendation[recomendation['Калорийность'].apply(lambda x: x >= cals)]
        
    if fat is not None:
        recomendation = recomendation[recomendation['Жирность'].apply(term_to_fat[fat])]

    if taste is not None: 
        recomendation = recomendation[recomendation['Вкус'].apply(lambda x: taste in x.split(' '))]
    
    if is_vegan is not None:
        recomendation = recomendation[recomendation['Вегетарианское'].apply(lambda x: x == is_vegan)]
    
    if method is not None:
        recomendation = recomendation[recomendation['Приготовления'].apply(lambda x: x == method)]

    return recomendation

In [ ]:
def do_fliter_2(recomendation, filters: list):
    for f in filters:
        c, f = f[0], f[1]

        if c == 'taste':
            def filter(value):
                value = value.split(' ')
                for i in value:
                    if f(i):
                        return True
                return False 

            recomendation = recomendation[recomendation['Вкус'].apply(filter)]
    
    return recomendation

In [ ]:
import re
from copy import deepcopy

class Request:
    like = None
    dislike = None
    
    filters = []

    @staticmethod
    def resolve_filters(filters_str: str) -> list:
        filters = []
        
        neg = False

        with_prev = False

        def norm_taste(taste):
            if taste[-2:] == 'ие':
                taste = taste[:-2] + 'ое'
            elif taste[-2:] == 'ые':
                taste = taste[:-2] + 'ое'

            return taste.lower()

        while len(filters_str) != 0:
            # пробел
            ###########################################################################
            space = re.match(r'(\s+)', filters_str)
            if space is not None:
                space = space.group(1)
                filters_str = filters_str[len(space):]

                with_prev = True

                continue

            # и
            ###########################################################################
            i = re.match(r'(и\s?)', filters_str)
            if i is not None:
                i = i.group(1)
                filters_str = filters_str[len(i):]
                
                neg = False

                continue

            # не
            ###########################################################################
            inverse = re.match(r'(не\s?)', filters_str)
            if inverse is not None:
                inverse = inverse.group(1)
                filters_str = filters_str[len(inverse):]

                continue

            # вкус
            ###########################################################################
            taste = re.match(
                r'('
                    r'сладк(ие|ое)|'
                    r'солен(ые|ое)|'
                    r'кисл(ые|ое)|'
                    r'остр(ые|ое)|'
                    r'нейтральн(ые|ое)|'
                    r'горьк(ие|ое)'
                r')',
                filters_str,
                re.IGNORECASE
            )
            if taste is not None:
                taste = taste.group(1)
                filters_str = filters_str[len(taste):]

                value = norm_taste(taste)

                f = lambda x, v=value: x.lower() == v

                if neg:
                    neg = False

                    f = lambda x, r=f: not r(x)

                if with_prev:
                    with_prev = False

                    if filters[-1][0] == 'taste':
                        l = filters[-1][1]
                        f = lambda x, r=f: r(x) or l(x)

                        filters.pop()
                    
                filters.append(('taste', f))

                continue

                # категория
                ###########################################################################

        return filters
        

user_text = 'Покажи все сладкие и соленые и из фруктов блюда'
while user_text != 'спасибо':
    request = Request()

    # user_text = input()

    result = re.match(r'(покажи все|выдай все|какие есть)\s(.*\s?)(блюда)', user_text, re.IGNORECASE)
    if result:
        filters_str = result.group(2)

        request.filters = Request.resolve_filters(filters_str)
    
    break

In [ ]:
recomendation = get_recomendation(['Салат с рукколой'], [])
recomendation = do_fliter_2(recomendation, request.filters)
recomendation

# request.filters[0][1]('соленое')